In [7]:
import numpy as np
from music21 import converter, instrument, stream, note, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Carregar e pré-processar dados MIDI
def prepare_sequences(notes, n_vocab):
    sequence_length = 100

    # Mapear notas para números
    pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input, network_output = [], []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)

    network_output = np.array(network_output)

    return network_input, network_output

# Criar o modelo LSTM
def create_network(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    model.add(LSTM(256))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(n_vocab, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

    return model

# Gerar novas melodias
def generate_notes(model, network_input, pitchnames, n_vocab):
    start = np.random.randint(0, len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    for note_index in range(1000):  # Você pode ajustar o número de notas geradas
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

# Carregar dados MIDI
midi_path = './SweetDreams.mid'
midi_stream = converter.parse(midi_path)

notes = []
for element in midi_stream.flat:
    if isinstance(element, note.Note):
        notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

# Preparar sequências para o modelo
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

# Criar e treinar o modelo
model = create_network(network_input, n_vocab)
model.fit(network_input, network_output, epochs=50, batch_size=64)

# Obter os nomes das notas a partir dos dados MIDI
pitchnames = sorted(set(item for item in notes))

# Gerar novas melodias
generated_notes = generate_notes(model, network_input, pitchnames, n_vocab)

# Criar um stream MIDI a partir das notas geradas
offset = 0
output_notes = []

for pattern in generated_notes:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    offset += 0.5  # Você pode ajustar o espaçamento entre as notas geradas

# Salvar as notas geradas em um novo arquivo MIDI
midi_stream_generated = stream.Stream(output_notes)
midi_stream_generated.write('midi', fp='output_generated.mid')



Epoch 1/50
13/13 [==============================] - 12s 599ms/step - loss: 3.9554
Epoch 2/50
13/13 [==============================] - 8s 595ms/step - loss: 3.7125
Epoch 3/50
13/13 [==============================] - 7s 519ms/step - loss: 3.6822
Epoch 4/50
13/13 [==============================] - 7s 545ms/step - loss: 3.6743
Epoch 5/50
13/13 [==============================] - 8s 615ms/step - loss: 3.6738
Epoch 6/50
13/13 [==============================] - 7s 533ms/step - loss: 3.6722
Epoch 7/50
13/13 [==============================] - 7s 519ms/step - loss: 3.6644
Epoch 8/50
13/13 [==============================] - 7s 530ms/step - loss: 3.6647
Epoch 9/50
13/13 [==============================] - 7s 515ms/step - loss: 3.6681
Epoch 10/50
13/13 [==============================] - 7s 576ms/step - loss: 3.6682
Epoch 11/50
13/13 [==============================] - 7s 499ms/step - loss: 3.6687
Epoch 12/50
13/13 [==============================] - 7s 529ms/step - loss: 3.6628
Epoch 13/50
13/13 [=====

'output_generated.mid'